In [1]:
USER_FLAG = "--user"
!pip install {USER_FLAG} --upgrade "tfx[kfp]<2"
!pip install {USER_FLAG} apache-beam[interactive]

  Using cached tensorflow-2.10.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (578.1 MB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.7/site-packages/jupyter_client-7.4.8.dist-info/METADATA'

  Using cached jupyter_client-6.1.12-py3-none-any.whl (112 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import os
import IPython
if not os.getenv(""):
    IPython.Application.instance().kernel.do_shutdown(True)

In [4]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tfx import v1 as tfx
import kfp
print('TensorFlow version:', tf.__version__)
print('TFX version: ',tfx.__version__)
print('KFP version: ',kfp.__version__)

TensorFlow version: 2.7.4
TFX version:  1.11.0
KFP version:  1.8.17


In [54]:
PROJECT_ID="vertex-ai-gcp-1"
!gcloud config set project {PROJECT_ID}
BUCKET_NAME="tfx_pipeline_demo"
NAME_PIPELINE = "tfx-pipeline"
ROOT_PIPELINE = f'gs://{BUCKET_NAME}/root/{NAME_PIPELINE}'
MODULE_FOLDER = f'gs://{BUCKET_NAME}/module/{NAME_PIPELINE}'
OUTPUT_MODEL_DIR=f'gs://{BUCKET_NAME}/output_model/{NAME_PIPELINE}'
INPUT_DATA_DIR = 'gs://tfx_pipeline_input_data'

Updated property [core/project].


In [17]:
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
#from tfx.components.statistics_gen.component import StatisticsGen
#from tfx.components.schema_gen.component import SchemaGen
#from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx_bsl.public import tfxio
from tfx.components.base import executor_spec
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
#from tfx.components.example_validator.component import ExampleValidator
#from tfx.components import Transform
from tensorflow_metadata.proto.v0 import schema_pb2
import os

from typing import List
#from absl import logging

In [7]:
context_in = InteractiveContext()
example_gen_csv = tfx.components.CsvExampleGen(input_base=INPUT_DATA_DIR)
context_in.run(example_gen_csv)


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [9]:
gen_statistics = tfx.components.StatisticsGen(examples=example_gen_csv.outputs['examples'])
context_in.run(gen_statistics)
context_in.show(gen_statistics.outputs['statistics'])

In [11]:
gen_schema = tfx.components.SchemaGen(statistics=gen_statistics.outputs['statistics'])
context_in.run(gen_schema)
context_in.show(gen_schema.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'AF3',FLOAT,required,,-
'AF4',FLOAT,required,,-
'F3',FLOAT,required,,-
'F4',FLOAT,required,,-
'F7',FLOAT,required,,-
'F8',FLOAT,required,,-
'FC5',FLOAT,required,,-
'FC6',FLOAT,required,,-
'O1',FLOAT,required,,-


In [12]:
stats_validate = tfx.components.ExampleValidator(statistics=gen_statistics.outputs['statistics'],schema=gen_schema.outputs['schema'])
context_in.run(stats_validate)
context_in.show(stats_validate.outputs['anomalies'])

In [14]:
TRANSFORM_MODULE_PATH = 'file_transform.py'

In [15]:
%%writefile {TRANSFORM_MODULE_PATH}

import tensorflow as tf
import tensorflow_transform as tft

NAMES = ['AF3','F7','F3','FC5','T7','P7','O1','O2','P8','T8','FC6','F4','F8','AF4']
LABEL = 'eyeDetection'

def preprocessing_fn(raw_inputs):
    processed_data = dict()
    for items in NAMES:
        processed_data[items]=raw_inputs[items]
    processed_data[LABEL] = raw_inputs[LABEL]

    return processed_data

Overwriting file_transform.py


In [16]:
!gsutil cp file_transform.py {MODULE_FOLDER}/

Copying file://file_transform.py [Content-Type=text/x-python]...
/ [1 files][  376.0 B/  376.0 B]                                                
Operation completed over 1 objects/376.0 B.                                      


In [19]:
transform_data = tfx.components.Transform(
    examples=example_gen_csv.outputs['examples'],schema=gen_schema.outputs['schema'],
    module_file=os.path.join(MODULE_FOLDER, TRANSFORM_MODULE_PATH))
context_in.run(transform_data, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying file_transform.py -> build/lib
installing to /tmp/tmpo9x7tccg
running install
running install_lib
copying build/lib/file_transform.py -> /tmp/tmpo9x7tccg
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpo9x7tccg/tfx_user_code_Transform-0.0+e51bd9d3e82f9d26a95a7578092002fd647283b811e9ad1c189f57e7aa496483-py3.7.egg-info
running install_scripts
creating /tmp/tmpo9x7tccg/tfx_user_code_Transform-0.0+e51bd9d3e

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


Processing /tmp/tfx-interactive-2022-12-16T14_11_44.287142-blx13e48/_wheels/tfx_user_code_Transform-0.0+e51bd9d3e82f9d26a95a7578092002fd647283b811e9ad1c189f57e7aa496483-py3-none-any.whl
Processing /tmp/tfx-interactive-2022-12-16T14_11_44.287142-blx13e48/_wheels/tfx_user_code_Transform-0.0+e51bd9d3e82f9d26a95a7578092002fd647283b811e9ad1c189f57e7aa496483-py3-none-any.whl
Processing /tmp/tfx-interactive-2022-12-16T14_11_44.287142-blx13e48/_wheels/tfx_user_code_Transform-0.0+e51bd9d3e82f9d26a95a7578092002fd647283b811e9ad1c189f57e7aa496483-py3-none-any.whl


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-12-16T14_11_44.287142-blx13e48/Transform/transform_graph/6/.temp_path/tftransform_tmp/66ffd08600ff49fe938131f5f685b2a3/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-12-16T14_11_44.287142-blx13e48/Transform/transform_graph/6/.temp_path/tftransform_tmp/66ffd08600ff49fe938131f5f685b2a3/assets


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [36]:
train_sam = os.path.join(transform_data.outputs['transformed_examples'].get()[0].uri,'Split-train')
filenames_tfr = [os.path.join(train_sam, name) for name in os.listdir(train_sam)]
dataset = tf.data.TFRecordDataset(filenames_tfr, compression_type='GZIP')
for record in dataset.take(1):
    sample = tf.train.Example()
    sample.ParseFromString(record.numpy())
    print(sample)

features {
  feature {
    key: "AF3"
    value {
      float_list {
        value: 4324.6201171875
      }
    }
  }
  feature {
    key: "AF4"
    value {
      float_list {
        value: 4384.10009765625
      }
    }
  }
  feature {
    key: "F3"
    value {
      float_list {
        value: 4293.85009765625
      }
    }
  }
  feature {
    key: "F4"
    value {
      float_list {
        value: 4279.490234375
      }
    }
  }
  feature {
    key: "F7"
    value {
      float_list {
        value: 4004.6201171875
      }
    }
  }
  feature {
    key: "F8"
    value {
      float_list {
        value: 4632.81982421875
      }
    }
  }
  feature {
    key: "FC5"
    value {
      float_list {
        value: 4148.72021484375
      }
    }
  }
  feature {
    key: "FC6"
    value {
      float_list {
        value: 4207.68994140625
      }
    }
  }
  feature {
    key: "O1"
    value {
      float_list {
        value: 4097.43994140625
      }
    }
  }
  feature {
    key: "O2"


In [48]:
%%writefile trainer.py
from typing import List
from absl import logging
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2
COL_NAMES=['AF3','F7','F3','FC5','T7','P7','O1','O2','P8','T8','FC6','F4','F8','AF4']
LABEL="eyeDetection"
BATCH_SIZE_TRAIN = 40
BATCH_SIZE_EVAL = 20
def _input_fn(files,accessor,transform_output,size) -> tf.data.Dataset:
    dataset = accessor.tf_dataset_factory(
        files,
        tfxio.TensorFlowDatasetOptions(batch_size=size),
        schema=transform_output.raw_metadata.schema)
    tft_layer = transform_output.transform_features_layer()
    def apply_transform_fn(raw_input_features):
        features_transformed = tft_layer(raw_input_features)
        label_transformed = features_transformed.pop(LABEL)
        return features_transformed, label_transformed
    return dataset.map(apply_transform_fn).repeat()

def _get_serve_tf_examples_fn(model, transform_output):
    model.tft_layer = transform_output.transform_features_layer()

    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")])
    def serve_tf_examples_fn(serialized_tf_sample):
        feature_spec_required = {
            k: v for k, v in transform_output.raw_feature_spec().items() if k != LABEL
        }

        parsed_features = tf.io.parse_example(
            serialized_tf_sample,
            feature_spec_required)
        transformed_feat = model.tft_layer(parsed_features)
        return model(transformed_feat)
    return serve_tf_examples_fn

def _make_keras_model() -> tf.keras.Model:

    inputs_layer = [keras.layers.Input(shape=(1,), name=n) for n in COL_NAMES]
    lay = keras.layers.concatenate(inputs_layer)
    lay = keras.layers.Dense(16, activation='relu')(lay)
    lay = keras.layers.Dense(4, activation='relu')(lay)
    lay = keras.layers.Dense(16, activation='relu')(lay)
    lay = keras.layers.Dense(4, activation='relu')(lay)
    outputs_layer = keras.layers.Dense(1)(lay)

    model_classification = keras.Model(inputs=inputs_layer, outputs=outputs_layer)
    opti=keras.optimizers.Adam(1e-2)
    los=tf.keras.losses.BinaryCrossentropy(from_logits=False)
    metr=[keras.metrics.BinaryAccuracy()]
    model_classification.compile(
        optimizer=opti,loss=los,metrics=metr)
    return model_classification

def run_fn(fn_args: tfx.components.FnArgs):
    tf_transform = tft.TFTransformOutput(fn_args.transform_output)
    train_samples = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform,
        size=BATCH_SIZE_TRAIN)
    eval_samples = _input_fn(
        fn_args.eval_files,
        fn_args.data_accessor,
        tf_transform,
        size=BATCH_SIZE_EVAL)

    model_classification = _make_keras_model()
    model_classification.fit(
        train_samples,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_samples,
        validation_steps=fn_args.eval_steps)
    sign = {
        "serving_default": _get_serve_tf_examples_fn(model_classification, tf_transform),
    }
    model_classification.save(fn_args.serving_model_dir, save_format='tf',signatures=sign)


Overwriting trainer.py


In [49]:
!gsutil cp trainer.py {MODULE_FOLDER}/

Copying file://trainer.py [Content-Type=text/x-python]...
/ [1 files][  3.2 KiB/  3.2 KiB]                                                
Operation completed over 1 objects/3.2 KiB.                                      


In [51]:
trainer_file="trainer.py"
trainer_file_path=os.path.join(MODULE_FOLDER, trainer_file)
model_trainer = tfx.components.Trainer(
    examples=example_gen_csv.outputs["examples"],
    transform_graph=transform_data.outputs["transform_graph"],
    train_args=tfx.proto.TrainArgs(num_steps=200),
    eval_args=tfx.proto.EvalArgs(num_steps=10),
    module_file=trainer_file_path,
)
context_in.run(model_trainer, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer.py -> build/lib
installing to /tmp/tmp1j9xrw2_
running install
running install_lib
copying build/lib/trainer.py -> /tmp/tmp1j9xrw2_
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmp1j9xrw2_/tfx_user_code_Trainer-0.0+24afec689508b37abbde3bba7d36c21060730cbbe40248f59143e0a2db52819f-py3.7.egg-info
running install_scripts
creating /tmp/tmp1j9xrw2_/tfx_user_code_Trainer-0.0+24afec689508b37abbde3bba7d36c21060730cbbe40

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


Processing /tmp/tfx-interactive-2022-12-16T14_11_44.287142-blx13e48/_wheels/tfx_user_code_Trainer-0.0+24afec689508b37abbde3bba7d36c21060730cbbe40248f59143e0a2db52819f-py3-none-any.whl
INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


200/200 [==============================] - 3s 11ms/step - loss: 6.8448 - binary_accuracy: 0.5562 - val_loss: 7.4040 - val_binary_accuracy: 0.5200
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-12-16T14_11_44.287142-blx13e48/Trainer/model/11/Format-Serving/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-12-16T14_11_44.287142-blx13e48/Trainer/model/11/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 11
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [52]:
def _create_pipeline(pl_name, pipeline_root_folder, data_root,
                     module_file_transform, module_file_train, model_dir_save,
                     ) -> tfx.dsl.Pipeline:

    example_gen_csv = tfx.components.CsvExampleGen(input_base=data_root)
    gen_statistics = tfx.components.StatisticsGen(examples=example_gen_csv.outputs['examples'])
    gen_schema = tfx.components.SchemaGen(statistics=gen_statistics.outputs['statistics'])
    transform_data = tfx.components.Transform(examples=example_gen_csv.outputs['examples'],schema=gen_schema.outputs['schema'],module_file=os.path.join(MODULE_FOLDER, TRANSFORM_MODULE_PATH))

    
    model_trainer = tfx.components.Trainer(
        module_file=module_file_train,
        examples=example_gen_csv.outputs['examples'],
        transform_graph=transform_data.outputs['transform_graph'],
        schema=gen_schema.outputs['schema'],
        train_args=tfx.proto.TrainArgs(num_steps=200),
        eval_args=tfx.proto.EvalArgs(num_steps=10))

    pusher = tfx.components.Pusher(
        model=model_trainer.outputs['model'],
        push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
        base_directory=model_dir_save)))

    return tfx.dsl.Pipeline(
        pipeline_name=pl_name,
        pipeline_root=pipeline_root_folder,
        components=[example_gen_csv,gen_statistics,gen_schema,transform_data,model_trainer,pusher])

In [58]:
trainer_file="trainer.py"
file_transform=os.path.join(MODULE_FOLDER, TRANSFORM_MODULE_PATH)
file_train=os.path.join(MODULE_FOLDER, trainer_file)
pl_def_file = NAME_PIPELINE + '.json'

pl_runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=pl_def_file)
# Following function will write the pipeline definition to PIPELINE_DEFINITION_FILE.
_ = pl_runner.run(
    _create_pipeline(
        pl_name=NAME_PIPELINE,
        pipeline_root_folder=ROOT_PIPELINE,
        data_root=INPUT_DATA_DIR,
        module_file_transform=file_transform,
        module_file_train=file_train,
        model_dir_save=OUTPUT_MODEL_DIR))

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying file_transform.py -> build/lib
installing to /tmp/tmpxo5m5_bs
running install
running install_lib
copying build/lib/file_transform.py -> /tmp/tmpxo5m5_bs
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpxo5m5_bs/tfx_user_code_Transform-0.0+e51bd9d3e82f9d26a95a7578092002fd647283b811e9ad1c189f57e7aa496483-py3.7.egg-info
running install_scripts
creating /tmp/tmpxo5m5_bs/tfx_user_code_Transform-0.0+e51bd9d3e

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer.py -> build/lib
installing to /tmp/tmpx1dtwdcn
running install
running install_lib
copying build/lib/trainer.py -> /tmp/tmpx1dtwdcn
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpx1dtwdcn/tfx_user_code_Trainer-0.0+24afec689508b37abbde3bba7d36c21060730cbbe40248f59143e0a2db52819f-py3.7.egg-info
running install_scripts
creating /tmp/tmpx1dtwdcn/tfx_user_code_Trainer-0.0+24afec689508b37abbde3bba7d36c21060730cbbe40

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
aiplatform.init(project=PROJECT_ID, location="us-central1")

job = pipeline_jobs.PipelineJob(template_path=pl_def_file,
                                display_name=PIPELINE_NAME)
job.run(sync=False)


In [62]:
import google.cloud
google.cloud.aiplatform.init(project=PROJECT_ID, location="us-central1")
job=google.cloud.aiplatform.pipeline_jobs.PipelineJob(template_path=pl_def_file,
                                display_name=NAME_PIPELINE)
job.run(sync=False)

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/953811166431/locations/us-central1/pipelineJobs/tfx-pipeline-20221216154720


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/953811166431/locations/us-central1/pipelineJobs/tfx-pipeline-20221216154720


To use this PipelineJob in another session:


...

INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/953811166431/locations/us-central1/pipelineJobs/tfx-pipeline-20221216154720 current state:
PipelineState.PIPELINE_STATE_RUNNING


PipelineJob projects/953811166431/locations/us-central1/pipelineJobs/tfx-pipeline-20221216154720 current state:
PipelineState.PIPELINE_STATE_RUNNING


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/953811166431/locations/us-central1/pipelineJobs/tfx-pipeline-20221216154720 current state:
PipelineState.PIPELINE_STATE_RUNNING


PipelineJob run completed. Resource name: projects/953811166431/locations/us-central1/pipelineJobs/tfx-pipeline-20221216154720


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob run completed. Resource name: projects/953811166431/locations/us-central1/pipelineJobs/tfx-pipeline-20221216154720
